In [2]:
import numpy as np
def func(x, y):
    return x*(1-x)*np.cos(4*np.pi*x) * np.sin(4*np.pi*y**2)**2

In [3]:
grid_x, grid_y = np.mgrid[0:1:100j, 0:1:200j]
points = np.random.rand(1000, 2)
values = func(points[:,0], points[:,1])
from scipy.interpolate import griddata
grid_z0 = griddata(points, values, (grid_x, grid_y), method='nearest')

In [7]:
gri

(1000, 2)

In [8]:
values.shape

(1000,)

In [9]:
grid_x.shape

(100, 200)

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
from datetime import datetime

In [4]:
now=datetime.now()

In [5]:
now

datetime.datetime(2019, 6, 19, 10, 4, 34, 22574)

In [6]:
now.year

2019

In [7]:
datetime.now()

datetime.datetime(2019, 6, 19, 10, 5, 19, 333601)

In [8]:
from datetime import timedelta

In [9]:
start=datetime(2016,10,1)

In [12]:
start+timedelta(hours=5)

datetime.datetime(2016, 10, 1, 5, 0)

In [13]:
datetime.fromtimestamp(0)

datetime.datetime(1970, 1, 1, 10, 0)

In [1]:
from parcels import FieldSet, ParticleSet, Variable, JITParticle, AdvectionRK4, plotTrajectoriesFile, ErrorCode
import numpy as np
import scipy as sc
import math
from datetime import timedelta, datetime
from operator import attrgetter
import copy_plottrajectoriesfile as cpt
import matplotlib.animation as animation
import imageio
import matplotlib.pyplot as plt
import scipy.io as sio #to load matlab file

"""
Load IMOS raw data
"""
filenames = "../raw_imos_current_data/IMOS_OceanCurrent_HV_2019_C-20190520T232835Z.nc"
variables = {'U' : 'UCUR',
                'V' : 'VCUR'}
dimensions = {'lat' : 'LATITUDE',
                  'lon' : 'LONGITUDE',
                  'time' : 'TIME'}
fieldset_imos = FieldSet.from_netcdf(filenames, variables, dimensions, deferred_load=False)

In [2]:

"""
Take TIME variable from the IMOS raw data
"""
from netCDF4 import Dataset
dataset=Dataset(filenames)
print(dataset.variables['TIME'])
time_imos=np.array(dataset.variables['TIME'])

<class 'netCDF4._netCDF4.Variable'>
float64 TIME(TIME)
    long_name: analysis time
    standard_name: time
    units: days since 1985-01-01 00:00:00+00:00
    coordinate_type: TIME
    calendar: gregorian
unlimited dimensions: TIME
current shape = (111,)
filling on, default _FillValue of 9.969209968386869e+36 used



In [3]:
"""
Change U and V shape 
"""

lati=fieldset_imos.U.lat
longi=fieldset_imos.U.lon
ucur=fieldset_imos.U.data
vcur=fieldset_imos.V.data
u_imos=[]
v_imos=[]
grid_imos=[[],[]]

for i in range(len(lati)): #Create grid of IMOS latitude longitude
    for j in range(len(longi)):
        grid_imos[0].append(lati[i])
        grid_imos[1].append(longi[j]) 
grid_imos=np.asarray(grid_imos).T

for t in range(len(ucur)): #for every time step
    values_u=[]
    values_v=[]
    for i in range(len(lati)): #take U and V for every couple (lati,longi)
        for j in range(len(longi)):
            values_u.append(ucur[t,i,j])
            values_v.append(vcur[t,i,j])
    u_imos.append(np.asarray(values_u))
    v_imos.append(np.asarray(values_v))


In [24]:
"""
time_imos float to datetime object
"""

time=[]
for i in range(len(time_imos)):
    time.append(datetime.fromtimestamp(0)+timedelta(days=time_imos[i])+(datetime(1985,1,1,0,0,0)-datetime.fromtimestamp(0)))


datetime.datetime(1985, 1, 1, 4, 0)

In [26]:
hey=[]
for i in range(len(temps)):
    hey.append(datetime.fromtimestamp(temps[i]))

In [28]:
for i in range(len(temps)):
    hey[i]+=timedelta(days=365.25*15)

In [29]:
hey

[datetime.datetime(1985, 1, 1, 7, 26, 58),
 datetime.datetime(1985, 1, 1, 7, 26, 59),
 datetime.datetime(1985, 1, 1, 7, 27),
 datetime.datetime(1985, 1, 1, 7, 27, 1),
 datetime.datetime(1985, 1, 1, 7, 27, 2),
 datetime.datetime(1985, 1, 1, 7, 27, 3),
 datetime.datetime(1985, 1, 1, 7, 27, 4),
 datetime.datetime(1985, 1, 1, 7, 27, 5),
 datetime.datetime(1985, 1, 1, 7, 27, 6),
 datetime.datetime(1985, 1, 1, 7, 27, 7),
 datetime.datetime(1985, 1, 1, 7, 27, 8),
 datetime.datetime(1985, 1, 1, 7, 27, 9),
 datetime.datetime(1985, 1, 1, 7, 27, 10),
 datetime.datetime(1985, 1, 1, 7, 27, 11),
 datetime.datetime(1985, 1, 1, 7, 27, 12),
 datetime.datetime(1985, 1, 1, 7, 27, 13),
 datetime.datetime(1985, 1, 1, 7, 27, 14),
 datetime.datetime(1985, 1, 1, 7, 27, 15),
 datetime.datetime(1985, 1, 1, 7, 27, 16),
 datetime.datetime(1985, 1, 1, 7, 27, 17),
 datetime.datetime(1985, 1, 1, 7, 27, 18),
 datetime.datetime(1985, 1, 1, 7, 27, 19),
 datetime.datetime(1985, 1, 1, 7, 27, 20),
 datetime.datetime(1985,